In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

In [2]:
pd.set_option('max_columns', 50)

In [3]:
df = gpd.read_file(
    'housing_sites/xn--Bay_Area_Housing_Opportunity_Sites_Inventory__20072023_-it38a.shp'
)

In [4]:
sf_sites = df.query('jurisdict == "San Francisco" and rhnacyc == "RHNA5"').copy()
sf_sites['allowden'] = sf_sites['allowden'].astype(float)
sf_sites['relcapcty'] = sf_sites['relcapcty'].astype(float)

In [7]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

to_plot = sf_sites.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(15, 15))

to_plot.plot(ax=ax, column='allowden', legend=True)

ctx.add_basemap(
    ax, 
    source='https://tiles.stadiamaps.com/tiles/osm_bright/{z}/{x}/{y}{r}.png?api_key=c50d11b9-261a-4916-b968-fe8854621c78'
)

In [8]:
df.columns

Index(['objectid', 'rhnacyc', 'rhnayrs', 'county', 'jurisdict', 'apn',
       'locapn', 'genplan', 'zoning', 'gacres', 'locacres', 'currunits',
       'allowden', 'allowlow', 'allowhigh', 'relcapcty', 'desafford', 'aff_vl',
       'aff_l', 'aff_m', 'aff_am', 'num_vl', 'num_l', 'num_m', 'num_am',
       'infcapcty', 'siteconst', 'sitetype', 'pdaparcel', 'bundled',
       'existuse', 'localnote', 'Shape__Are', 'Shape__Len', 'geometry'],
      dtype='object')

In [5]:
sf_building_permits = pd.read_csv('https://data.sfgov.org/api/views/p4e4-a5a7/rows.csv?accessType=DOWNLOAD')

/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (22,32,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
sf_building_permits['apn'] = sf_building_permits['Block'] + '/' + sf_building_permits['Lot']

In [30]:
sf_building_permits['Filed Date'] = pd.to_datetime(sf_building_permits['Filed Date'])

In [161]:
[c for c in sf_building_permits.columns if 'Date' in c]

['Permit Creation Date',
 'Current Status Date',
 'Filed Date',
 'Issued Date',
 'Completed Date',
 'First Construction Document Date',
 'Permit Expiration Date']

In [162]:
sf_building_permits['Current Status Date'] = pd.to_datetime(sf_building_permits['Current Status Date'])
sf_building_permits['Issued Date'] = pd.to_datetime(sf_building_permits['Issued Date'])
sf_building_permits['Completed Date'] = pd.to_datetime(sf_building_permits['Completed Date'])
sf_building_permits['First Construction Document Date'] = pd.to_datetime(sf_building_permits['First Construction Document Date'])
sf_building_permits['Permit Expiration Date'] = pd.to_datetime(sf_building_permits['Permit Expiration Date'])

In [163]:
sf_building_permits[[
    'Permit Type', 'Permit Type Definition'
]].drop_duplicates().sort_values('Permit Type')

Permit Type               Permit Type Definition
1402            1                     new construction
284             2          new construction wood frame
12              3     additions alterations or repairs
0               4                         sign - erect
3136            5  grade or quarry or fill or excavate
1289            6                          demolitions
17              7                 wall or painted sign
2               8               otc alterations permit

In [164]:
sf_building_permits.loc[lambda x: x['Permit Type'].isin([1, 2])]['Proposed Use'].value_counts()

apartments              774
1 family dwelling       521
2 family dwelling       347
office                   73
prkng garage/private     29
recreation bldg          17
tourist hotel/motel      15
storage shed             14
warehouse,no frnitur     12
school                   12
food/beverage hndlng     11
retail sales             10
manufacturing             7
prkng garage/public       7
workshop commercial       7
filling/service stn       6
misc group residns.       6
residential hotel         5
auto repairs              5
muni driver restroom      5
fence/retaining wall      5
health studios & gym      3
church                    3
clinics-medic/dental      3
warehouse, furniture      3
public assmbly other      3
radio & tv stations       2
sfpd or sffd station      2
car wash                  2
nursing home non amb      2
parking lot               2
power plant               2
consulate                 1
ambulance service         1
greenhouse                1
printing plant      

In [165]:
sf_new_construction_all = sf_building_permits[
    sf_building_permits['Permit Type'].isin([1, 2])
    & sf_building_permits['Proposed Use'].isin(['apartments', '1 family dwelling', '2 family dwelling', 'residential hotel', 'misc group residns.'])
]

In [181]:
sf_new_construction_post_2015 = sf_building_permits[
    sf_building_permits['Permit Type'].isin([1, 2])
    & (sf_building_permits['Issued Date'] >= '2015-01-01')
    & sf_building_permits['Proposed Use'].isin(['apartments', '1 family dwelling', '2 family dwelling', 'residential hotel', 'misc group residns.'])
]

In [182]:
pd.set_option('max_columns', 100)

## Make sure the `relcapcty` column is what we want:

In [183]:
test_df = pd.DataFrame({
    'maybe_density': sf_sites['allowden'] * sf_sites['locacres'],
    'capacity': sf_sites['relcapcty'].astype(float)
}).dropna()
test_df['maybe_density_int'] = test_df['maybe_density'].round(1)
test_df['capacity_int'] = test_df['capacity'].round(1)

In [184]:
(test_df['maybe_density_int'] == test_df['capacity_int']).mean()

1.0

## Now let's do some stats:

In [185]:
sf_sites['apn'].isin(sf_new_construction_all['apn']).mean()

0.06187073725235466

In [186]:
sf_sites['apn'].isin(sf_new_construction_post_2015['apn']).mean()

0.033290029230269566

In [187]:
sf_new_construction_all['apn'].isin(sf_sites['apn']).mean()

0.31518451300665457

In [188]:
sf_new_construction_post_2015['apn'].isin(sf_sites['apn']).mean()

0.3220338983050847

## Number of units in HE:

In [189]:
sf_sites['relcapcty'].sum()

47209.0

## Overall number of units completed:

In [190]:
sf_new_construction_all[
    sf_new_construction_all['Filed Date'].notnull()
]['Proposed Units'].sum()

51016.0

In [191]:
sf_new_construction_all[
    sf_new_construction_all['Issued Date'].notnull()
]['Proposed Units'].sum()

32460.0

In [192]:
sf_new_construction_all[
    sf_new_construction_all['First Construction Document Date'].notnull()
]['Proposed Units'].sum()

25780.0

In [193]:
sf_new_construction_all[
    sf_new_construction_all['Completed Date'].notnull()
]['Proposed Units'].sum()

13432.0

In [194]:
sf_new_construction_post_2015[
    sf_new_construction_post_2015['Filed Date'].notnull()
]['Proposed Units'].sum()

28137.0

In [195]:
sf_new_construction_post_2015[
    sf_new_construction_post_2015['Issued Date'].notnull()
]['Proposed Units'].sum()

28137.0

In [196]:
sf_new_construction_post_2015[
    sf_new_construction_post_2015['First Construction Document Date'].notnull()
]['Proposed Units'].sum()

21691.0

In [197]:
sf_new_construction_post_2015[
    sf_new_construction_post_2015['Completed Date'].notnull()
]['Proposed Units'].sum()

9445.0

In [198]:
sf_new_construction_all[
    sf_new_construction_all['Completed Date'] >= '2015-01-01'
]['Proposed Units'].sum()

13427.0

In [199]:
13427 / 28800

0.46621527777777777

In [200]:
13427 / 5

2685.4

In [201]:
sf_new_construction_all[
    (sf_new_construction_all['Completed Date'] >= '2015-01-01')
    & (sf_new_construction_all['Issued Date'] < '2015-01-01')
]

Permit Number  Permit Type       Permit Type Definition  \
284    201301047264            2  new construction wood frame   
1402   201301168124            1             new construction   
2690   201301319232            1             new construction   
2723   201301319232            1             new construction   
4138   201302150317            2  new construction wood frame   
...             ...          ...                          ...   
50430  201406057554            2  new construction wood frame   
50431  201406057554            2  new construction wood frame   
50432  201406057554            2  new construction wood frame   
50433  201406057555            2  new construction wood frame   
51343  201406138386            1             new construction   

      Permit Creation Date Block  Lot  Street Number Street Number Suffix  \
284             01/04/2013  6415  022           1310                    V   
1402            01/16/2013  3616  084           2558                  NaN   
2690            01/31/2013  3509  043            104                  NaN   
2723            01/31/2013  3509  043           1321                  NaN   
4138            02/15/2013  0776  035           1470                  NaN   
...                    ...   ...  ...            ...                  ...   
50430           06/05/2014  0629  037           2468                  NaN   
50431           06/05/2014  0629  037           2470                  NaN   
50432           06/05/2014  0629  037           2472                  NaN   
50433           06/05/2014  0629  037           2473                  NaN   
51343           06/13/2014  0281  003            832                  NaN   

      Street Name Street Suffix  Unit Unit Suffix  \
284        Geneva            Av   NaN         NaN   
1402      Mission            St   NaN         NaN   
2690         09th            St   NaN         NaN   
2723      Mission            St   NaN         NaN   
4138   Mcallister            St   NaN         NaN   
...           ...           ...   ...         ...   
50430  Sacramento            St   NaN         NaN   
50431  Sacramento            St   NaN         NaN   
50432  Sacramento            St   NaN         NaN   
50433        Clay            St   NaN         NaN   
51343      Sutter            St   NaN         NaN   

                                             Description Current Status  \
284    erect 3-story type v single family dwelling re...       complete   
1402   to erect 8 stories, 1 basement, 114 units resi...       complete   
2690   to erect 160 units, 11 stories with basement m...       complete   
2723   to erect 160 units, 11 stories with basement m...       complete   
4138   erect 4-story three dwelling units type v new ...       complete   
...                                                  ...            ...   
50430  (townhouse# 2) to erect 3 stories, no basement...       complete   
50431  (townhouse# 2) to erect 3 stories, no basement...       complete   
50432  (townhouse# 2) to erect 3 stories, no basement...       complete   
50433  (townhouse# 3) to erect 3 stories, no basement...       complete   
51343  to erect 5 stories, no basement, 20 units resi...       complete   

      Current Status Date Filed Date Issued Date Completed Date  \
284            2015-04-01 2013-01-04  2013-06-14     2015-04-01   
1402           2015-01-12 2013-01-16  2013-06-07     2015-01-12   
2690           2016-09-13 2013-01-31  2013-08-29     2016-09-13   
2723           2016-09-13 2013-01-31  2013-08-29     2016-09-13   
4138           2016-10-05 2013-02-15  2013-12-19     2016-10-05   
...                   ...        ...         ...            ...   
50430          2016-09-26 2014-06-05  2014-12-02     2016-09-26   
50431          2016-09-26 2014-06-05  2014-12-02     2016-09-26   
50432          2016-09-26 2014-06-05  2014-12-02     2016-09-26   
50433          2016-09-22 2014-06-05  2014-12-02     2016-09-22   
51343          2016-09-16

# By capacity

In [134]:
sf_sites.columns

Index(['objectid', 'rhnacyc', 'rhnayrs', 'county', 'jurisdict', 'apn',
       'locapn', 'genplan', 'zoning', 'gacres', 'locacres', 'currunits',
       'allowden', 'allowlow', 'allowhigh', 'relcapcty', 'desafford', 'aff_vl',
       'aff_l', 'aff_m', 'aff_am', 'num_vl', 'num_l', 'num_m', 'num_am',
       'infcapcty', 'siteconst', 'sitetype', 'pdaparcel', 'bundled',
       'existuse', 'localnote', 'Shape__Are', 'Shape__Len', 'geometry'],
      dtype='object')

In [135]:
sf_sites.shape

(6158, 35)

In [136]:
sf_sites['relcapcty'].value_counts()[:10]

1.0     1103
2.0      959
0.0      904
3.0      687
4.0      463
5.0      291
6.0      227
7.0      194
8.0      139
9.0      117
10.0     102
Name: relcapcty, dtype: int64

In [137]:
(sf_sites['relcapcty'] > 10).sum()

972

In [138]:
for n in range(1, 11):
    print(n)
    print(
        sf_sites[
            sf_sites['relcapcty'] == n
        ]['apn'].isin(sf_new_construction_all['apn']).mean()
    )
    print()

1
0.09247506799637353

2
0.054223149113660066

3
0.036390101892285295

4
0.06695464362850972

5
0.061855670103092786

6
0.039647577092511016

7
0.07731958762886598

8
0.07194244604316546

9
0.09401709401709402

10
0.029411764705882353



In [139]:
sf_sites[
    sf_sites['relcapcty'] > 10
]['apn'].isin(sf_new_construction_all['apn']).mean()

0.09670781893004116

# Merging the datasets

In [140]:
merged_df = sf_sites.merge(
    sf_new_construction_all,
    on='apn',
    how='left',
    indicator=True
)

In [141]:
merged_df['Lot Size Sq Ft'] = merged_df['locacres'] * 43560.

In [142]:
increase_df = merged_df[
    merged_df['Proposed Units'] > merged_df['relcapcty']
]

In [143]:
pd.set_option('max_rows', 100)

In [144]:
increase_df[
    increase_df['relcapcty'] == 0
]

objectid rhnacyc    rhnayrs county      jurisdict        apn     locapn  \
15       66781   RHNA5  2015-2023   6075  San Francisco   0024/020   0024/020   
16       66781   RHNA5  2015-2023   6075  San Francisco   0024/020   0024/020   
766      70115   RHNA5  2015-2023   6075  San Francisco   0106/002   0106/002   
1736     71104   RHNA5  2015-2023   6075  San Francisco   3552/012   3552/012   
2508     71866   RHNA5  2015-2023   6075  San Francisco  5463/001G  5463/001G   
2509     71866   RHNA5  2015-2023   6075  San Francisco  5463/001G  5463/001G   
3294     72643   RHNA5  2015-2023   6075  San Francisco   1269/167   1269/167   
3621     72967   RHNA5  2015-2023   6075  San Francisco   2627/005   2627/005   
3622     72967   RHNA5  2015-2023   6075  San Francisco   2627/005   2627/005   
3843     73183   RHNA5  2015-2023   6075  San Francisco   5577/006   5577/006   
3844     73183   RHNA5  2015-2023   6075  San Francisco   5577/006   5577/006   
4530     73988   RHNA5  2015-2023   6075  San Francisco   4353/008   4353/008   
4853     74291   RHNA5  2015-2023   6075  San Francisco   5260/036   5260/036   
5019     74473   RHNA5  2015-2023   6075  San Francisco  1704/034A  1704/034A   
5020     74473   RHNA5  2015-2023   6075  San Francisco  1704/034A  1704/034A   
5502     75904   RHNA5  2015-2023   6075  San Francisco   4167/011   4167/011   

               genplan   zoning  gacres  locacres  currunits  allowden  \
15           Northeast      C-2   0.036  0.036468          0       0.0   
16           Northeast      C-2   0.036  0.036468          0       0.0   
766          Northeast      C-2   0.061  0.061228          0       0.0   
1736           Mission  PDR-1-G   0.666  0.666168          0       0.0   
2508  Other S Bayshore     RH-1   0.034  0.033752          0       0.0   
2509  Other S Bayshore     RH-1   0.034  0.033752          0       0.0   
3294       Buena Vista     RH-2   0.033  0.032640          0       0.0   
3621       Buena Vista     RH-2   0.033  0.033129          0       0.0   
3622       Buena Vista     RH-2   0.033  0.033129          0       0.0   
3843    Bernal Heights     RH-1   0.032  0.032391          0       0.0   
3844    Bernal Heights     RH-1   0.032  0.032391          0       0.0   
4530     BVHP Area A,B    PDR-2   0.847  0.847320          0       0.0   
4853  Other S Bayshore      M-1   0.054  0.054448          0       0.0   
5019      Outer Sunset     RH-2   0.033  0.033351          0       0.0   
5020      Outer Sunset     RH-2   0.033  0.033351          0       0.0   
5502    Showpl/Potrero  PDR-1-G   0.503  0.502838          0       0.0   

      allowlow  allowhigh  relcapcty desafford aff_vl aff_l aff_m aff_am  \
15           0          0        0.0      None   None  None  None   None   
16           0          0        0.0      None   None  None  None   None   
766          0          0        0.0      None   None  None  None   None   
1736         0          0        0.0      None   None  None  None   None   
2508         0          0        0.0      None   None  None  None   None   
2509         0          0        0.0      None   None  None  None   None   
3294         0          0        0.0      None   None  None  None   None   
3621         0          0        0.0      None   None  None  None   None   
3622         0          0        0.0      None   None  None  None   None   
3843         0          0        0.0      None   None  None  None   None   
3844         0          0        0.0      None   None  None  None   None   
4530         0          0        0.0      None   None  None  None   None   
4853         0          0        0.0      None   None  None  None   None   
5019         0          0        0.0      None   None  None  None   None   
5020         0          0        0.0      None   None  None  None   None   
5502         0          0        0.0      None   None  None  None   None   

      num_vl  num_l  num_m  num_am infcapcty siteconst       sitetype  \
15 

In [111]:
increase_df['relcapcty'].value_counts().sort_index()

0.0      14
1.0      31
2.0      16
3.0      21
4.0      17
5.0      13
6.0       7
7.0      14
8.0       4
9.0       9
10.0      1
11.0      5
12.0      4
13.0      3
14.0      4
15.0      3
16.0      1
17.0      1
18.0      3
19.0      1
20.0      2
21.0      4
22.0      2
23.0      2
24.0      4
25.0      1
26.0      2
27.0      2
29.0      1
33.0      1
34.0      1
35.0      1
36.0      2
38.0      1
39.0      2
40.0      1
42.0      1
43.0      2
45.0      3
46.0      2
47.0      1
53.0      1
57.0      1
58.0      1
59.0      2
65.0      2
66.0      1
67.0      5
69.0      6
72.0      1
73.0      1
75.0      3
77.0      1
78.0      1
83.0      2
88.0      1
97.0      1
108.0     2
118.0     1
122.0     1
141.0     1
154.0     1
Name: relcapcty, dtype: int64